# Import modules

In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import warnings

from datetime import date
from itertools import product
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

warnings.simplefilter('ignore')

/opt/conda/lib/python3.8/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Config

In [2]:
DATA_FILE_PATHS = '/workspace/rahnemacollege/Project/Git/demand-prediction/data/input/'

start_date = '2023-01-01'
start_date_test = '2023-04-01'
end_date = '2023-05-01'

FEATURE_LIST = [
    'PULocationID',
    'PU_day_of_month',
    'PU_day_of_week',
    'last_day_demand',
    'last_week_demand'
]

TARGET = 'count'
VALIDATION_SPLIT_RATIO = 0.2

LR_OUTPUT_PATH = '/workspace/rahnemacollege/Project/Git/demand-prediction/data/output/lr_result.parquet'
XGB_OUTPUT_PATH = '/workspace/rahnemacollege/Project/Git/demand-prediction/data/output/XGB_result.parquet'

# Load data

In [3]:
def load_data(file_paths, start_date=None, end_date=None):
    df = pd.read_parquet(file_paths)
    df['date'] = df['tpep_pickup_datetime'].dt.date.astype(str)

    if start_date:
        if end_date:
            df = df[(df['date'] >= start_date) & (
                df['date'] < end_date)]
        else:
            df = df[df['date'] > start_date].reset_index(drop=True)
    # Sort the DataFrame based on the 'tpep_pickup_datetime' column in ascending order
    df = df.sort_values(by='date')
    df = df.reset_index(drop=True)

    # Calculate the start time of each interval
    df['interval_start'] = df['tpep_pickup_datetime'].dt.floor('3H')

    # Calculate the end time of each interval
    df['interval_end'] = df['interval_start'] + pd.Timedelta(hours=3)

    # Create a new column with the time interval in the desired format
    df['time_interval'] = df['interval_start'].dt.strftime(
        '%H:%M:%S') + ' - ' + df['interval_end'].dt.strftime('%H:%M:%S')

    # Drop 'interval_start' and 'interval_end' columns
    df.drop(columns=['interval_start', 'interval_end'], inplace=True)

    # Create bins for interval numbers from 1 to 8
    df['time_interval_number'] = pd.cut(
        df['tpep_pickup_datetime'].dt.hour, bins=8, labels=range(1, 9), right=False)

    return df


rides_df = load_data(DATA_FILE_PATHS, start_date, end_date)
print(rides_df.shape)
rides_df.head()

(12672629, 22)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,date,time_interval,time_interval_number
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,...,0.5,0.00,0.00,1.0,14.30,2.5,0.00,2023-01-01,00:00:00 - 03:00:00,1
1,1,2023-01-01 16:18:55,2023-01-01 16:26:09,3.0,0.00,1.0,N,107,90,1,...,0.5,2.10,0.00,1.0,12.60,2.5,0.00,2023-01-01,15:00:00 - 18:00:00,6
2,2,2023-01-01 16:59:08,2023-01-01 17:41:59,1.0,19.82,2.0,N,132,238,1,...,0.5,16.36,6.55,1.0,98.16,2.5,1.25,2023-01-01,15:00:00 - 18:00:00,6
3,2,2023-01-01 16:29:59,2023-01-01 16:59:01,1.0,9.36,1.0,N,138,68,1,...,0.5,11.80,6.55,1.0,70.80,2.5,1.25,2023-01-01,15:00:00 - 18:00:00,6
4,2,2023-01-01 16:35:44,2023-01-01 16:53:56,2.0,3.18,1.0,N,114,162,1,...,0.5,4.76,0.00,1.0,28.56,2.5,0.00,2023-01-01,15:00:00 - 18:00:00,6


## Labeling

### Aggregate data and labeling

In [4]:
def labeling_by_interval(rides_df: pd.DataFrame):
    aggregated_df = rides_df.groupby(
        ['date', 'time_interval_number', 'PULocationID']).size().reset_index(name='count')
    unique_dates = rides_df['date'].unique()
    unique_interval = rides_df['time_interval_number'].unique()
    unique_pu_location_ids = rides_df['PULocationID'].unique()
    all_combinations = list(
        product(unique_dates, unique_interval, unique_pu_location_ids))
    combinations_df = pd.DataFrame(all_combinations, columns=[
                                   'date', 'time_interval_number', 'PULocationID'])
    label_df = aggregated_df.merge(combinations_df, how='right', on=[
                                   'date', 'time_interval_number', 'PULocationID']).fillna(0)
    # Sort based on two columns: 'time_interval_number' (ascending) and 'date' (ascending)
    label_df = label_df.sort_values(
        by=['date', 'time_interval_number'], ascending=[True, True])
    return label_df

In [5]:
labels_time_df = labeling_by_interval(rides_df)
print(labels_time_df.shape)
labels_time_df.head()

(251520, 4)


,date,time_interval_number,PULocationID,count
0,2023-01-01,1,161,504
1,2023-01-01,1,107,604
2,2023-01-01,1,132,389
3,2023-01-01,1,138,42
4,2023-01-01,1,114,205


## Feature Selection

### Add calender and previous demands features

In [6]:
def adding_feature(rides_df: pd.DataFrame):
    rides_df['date'] = rides_df['date'].astype('datetime64[ns]')
    rides_df['PU_day_of_month'] = rides_df['date'].dt.day.astype(np.uint8)
    rides_df['PU_day_of_week'] = rides_df['date'].dt.weekday.astype(np.uint8)
    rides_df = rides_df.sort_values(
        ['PULocationID', 'date', 'time_interval_number'])
    rides_df['last_interval_demand'] = rides_df.groupby(['PULocationID'])[
        'count'].shift(1)
    rides_df['last_day_demand'] = rides_df.groupby(['PULocationID'])[
        'count'].shift(8)
    rides_df['last_week_demand'] = rides_df.groupby(['PULocationID'])[
        'count'].shift(56)

    # To handle the issue with MAPE and zero values in the count column, we replace zeros
    # with the one value. This prevents division by zero and ensures
    # accurate evaluation of the metric.
    rides_df.loc[rides_df['count']
                 == 0, 'count'] = 1

    return rides_df


labels_time_df_feat = adding_feature(labels_time_df)
print(labels_time_df_feat.shape)
labels_time_df_feat.head()

(251520, 9)


,date,time_interval_number,PULocationID,count,PU_day_of_month,PU_day_of_week,last_interval_demand,last_day_demand,last_week_demand
58,2023-01-01,1,1,1,1,6,NaN,NaN,NaN
1368,2023-01-01,2,1,1,1,6,0.0,NaN,NaN
1630,2023-01-01,3,1,1,1,6,1.0,NaN,NaN
1892,2023-01-01,4,1,1,1,6,1.0,NaN,NaN
844,2023-01-01,5,1,13,1,6,1.0,NaN,NaN


In [7]:
rides_df = labels_time_df_feat

## Dropping some samples

In [8]:
rides_df = rides_df.dropna()

## Train and Test split

In [9]:
def train_and_test_split(data, split_date):
    train_data = data[
        rides_df['date'] < split_date
    ]
    test_data = data[
        rides_df['date'] >= split_date
    ]

    train_data.set_index('date', inplace=True)
    test_data.set_index('date', inplace=True)

    return train_data, test_data

In [10]:
train_df, test_df = train_and_test_split(
    rides_df,
    start_date_test
)

In [11]:
print(train_df.shape)
train_df.head()

(173968, 8)


,time_interval_number,PULocationID,count,PU_day_of_month,PU_day_of_week,last_interval_demand,last_day_demand,last_week_demand
date,,,,,,,,
2023-01-08,1,1,1,8,6,0.0,0.0,0.0
2023-01-08,2,1,1,8,6,0.0,0.0,1.0
2023-01-08,3,1,3,8,6,0.0,1.0,1.0
2023-01-08,4,1,1,8,6,3.0,1.0,1.0
2023-01-08,5,1,2,8,6,1.0,1.0,13.0


In [12]:
print(test_df.shape)
test_df.head()

(62880, 8)


,time_interval_number,PULocationID,count,PU_day_of_month,PU_day_of_week,last_interval_demand,last_day_demand,last_week_demand
date,,,,,,,,
2023-04-01,1,1,1,1,5,0.0,0.0,0.0
2023-04-01,2,1,1,1,5,0.0,0.0,0.0
2023-04-01,3,1,1,1,5,1.0,0.0,3.0
2023-04-01,4,1,1,1,5,1.0,2.0,2.0
2023-04-01,5,1,1,1,5,0.0,5.0,3.0


## Target and Feature split

In [13]:
train_label_df = train_df[TARGET]
train_df = train_df.drop(TARGET, axis=1)

test_label_df = test_df[TARGET]
test_df = test_df.drop(TARGET, axis=1)

In [14]:
print(train_df.shape)
train_df.head()

(173968, 7)


,time_interval_number,PULocationID,PU_day_of_month,PU_day_of_week,last_interval_demand,last_day_demand,last_week_demand
date,,,,,,,
2023-01-08,1,1,8,6,0.0,0.0,0.0
2023-01-08,2,1,8,6,0.0,0.0,1.0
2023-01-08,3,1,8,6,0.0,1.0,1.0
2023-01-08,4,1,8,6,3.0,1.0,1.0
2023-01-08,5,1,8,6,1.0,1.0,13.0


In [15]:
print(train_label_df.shape)
train_label_df.head()

(173968,)


date
2023-01-08    1
2023-01-08    1
2023-01-08    3
2023-01-08    1
2023-01-08    2
Name: count, dtype: int64

In [16]:
print(test_df.shape)
test_df.head()

(62880, 7)


,time_interval_number,PULocationID,PU_day_of_month,PU_day_of_week,last_interval_demand,last_day_demand,last_week_demand
date,,,,,,,
2023-04-01,1,1,1,5,0.0,0.0,0.0
2023-04-01,2,1,1,5,0.0,0.0,0.0
2023-04-01,3,1,1,5,1.0,0.0,3.0
2023-04-01,4,1,1,5,1.0,2.0,2.0
2023-04-01,5,1,1,5,0.0,5.0,3.0


In [17]:
print(test_label_df.shape)
test_label_df.head()

(62880,)


date
2023-04-01    1
2023-04-01    1
2023-04-01    1
2023-04-01    1
2023-04-01    1
Name: count, dtype: int64

## Train and Validation split

In [18]:
train_df, validation_df, train_label_df, validation_label_df = train_test_split(
    train_df,
    train_label_df,
    test_size=VALIDATION_SPLIT_RATIO,
    shuffle=False
)

# ML Models

In [19]:
def model_training(ml_model, train_df, train_label_df, **params):
    model = ml_model(**params)
    model.fit(
        train_df,
        train_label_df
    )
    return model


replace_negatives = np.vectorize(lambda x: 0 if x < 0 else x)

## Calculate Error

In [20]:
def symmetric_mean_absolute_percentage_error(actual, predicted) -> float:
    return round(
        np.mean(
            np.abs(predicted - actual) /
            ((np.abs(predicted) + np.abs(actual)) / 2)
        ), 4
    )


def error_calculator(real_demand, predicted_demand):
    print(
        'SMAPE: ',
        round(
            symmetric_mean_absolute_percentage_error(
                real_demand,
                predicted_demand
            ) * 100, 2
        ), '%'
    )
    print(
        'MAPE:  ',
        round(
            float(
                mean_absolute_percentage_error(
                    real_demand,
                    predicted_demand
                )
            ) * 100, 2
        ), '%'
    )
    print(
        'MSE:   ',
        round(
            float(
                mean_squared_error(
                    real_demand,
                    predicted_demand
                )
            ), 2
        )
    )
    print(
        'MAE:   ',
        round(
            float(
                mean_absolute_error(
                    real_demand,
                    predicted_demand
                )
            ), 2
        )
    )

## Linear Regression Model

In [21]:
lr_model = model_training(
    LinearRegression,
    train_df,
    train_label_df
)

### Validation prediction

In [22]:
lr_validation_pred = replace_negatives(
    np.round_(
        lr_model.predict(
            validation_df
        )
    )
)
error_calculator(
    validation_label_df,
    lr_validation_pred
)

SMAPE:  58.53 %
MAPE:   58.9 %
MSE:    1161.5
MAE:    12.0


### Test prediction

In [23]:
lr_test_pred = replace_negatives(
    np.round_(
        lr_model.predict(
            test_df
        )
    )
)
error_calculator(
    test_label_df,
    lr_test_pred
)

SMAPE:  70.52 %
MAPE:   63.03 %
MSE:    592.9
MAE:    7.65


### Result Data

In [24]:
lr_result_df = test_df.copy()
lr_result_df['real demand'] = test_label_df
lr_result_df['predicted demand'] = lr_test_pred

print(lr_result_df.shape)
lr_result_df.head()

(62880, 9)


,time_interval_number,PULocationID,PU_day_of_month,PU_day_of_week,last_interval_demand,last_day_demand,last_week_demand,real demand,predicted demand
date,,,,,,,,,
2023-04-01,1,1,1,5,0.0,0.0,0.0,1,0.0
2023-04-01,2,1,1,5,0.0,0.0,0.0,1,0.0
2023-04-01,3,1,1,5,1.0,0.0,3.0,1,3.0
2023-04-01,4,1,1,5,1.0,2.0,2.0,1,3.0
2023-04-01,5,1,1,5,0.0,5.0,3.0,1,4.0


In [25]:
lr_result_df.to_parquet(LR_OUTPUT_PATH)

## XGBoost Model

### Hyperparameter tuning

In [26]:
def hyper_parameter_tuning(n_estimators, learning_rate, max_depth, scoring_method):
    parameters = {
        'n_estimators': n_estimators,
        'learning_rate': learning_rate,
        'max_depth': max_depth
    }

    gc = GridSearchCV(
        XGBRegressor(tree_method='gpu_hist', gpu_id=1),
        parameters,
        scoring=scoring_method
    )

    gc.fit(
        train_df,
        train_label_df
    )

    param = gc.best_params_

    return param


n_estimators = [700, 1000]
learning_rate = [0.15, 0.1, 0.01]
max_depth = [1, 2, 3]
scoring_method = 'neg_root_mean_squared_error'

param = hyper_parameter_tuning(
    n_estimators,
    learning_rate,
    max_depth,
    scoring_method
)

print(param)

{'learning_rate': 0.1, 'max_depth': 1, 'n_estimators': 1000}


### XGBoost Model

In [27]:
XGB_model = model_training(
    XGBRegressor,
    train_df,
    train_label_df,
    n_estimators=param['n_estimators'],
    learning_rate=param['learning_rate'],
    max_depth=param['max_depth'],
    tree_method='gpu_hist',
    gpu_id=1
)

### Validation prediction

In [28]:
XGB_validation_pred = replace_negatives(
    np.round_(
        XGB_model.predict(
            validation_df
        )
    )
)
error_calculator(
    validation_label_df,
    XGB_validation_pred
)

SMAPE:  54.54 %
MAPE:   61.19 %
MSE:    1327.64
MAE:    12.68


### Test prediction

In [29]:
XGB_test_pred = replace_negatives(
    np.round_(
        XGB_model.predict(
            test_df
        )
    )
)
error_calculator(
    test_label_df,
    XGB_test_pred
)

SMAPE:  62.14 %
MAPE:   66.53 %
MSE:    701.13
MAE:    8.06


### Result Data

In [30]:
XGB_result_df = test_df.copy()
XGB_result_df['real demand'] = test_label_df
XGB_result_df['predicted demand'] = XGB_test_pred

print(XGB_result_df.shape)
XGB_result_df.head()

(62880, 9)


,time_interval_number,PULocationID,PU_day_of_month,PU_day_of_week,last_interval_demand,last_day_demand,last_week_demand,real demand,predicted demand
date,,,,,,,,,
2023-04-01,1,1,1,5,0.0,0.0,0.0,1,0.0
2023-04-01,2,1,1,5,0.0,0.0,0.0,1,0.0
2023-04-01,3,1,1,5,1.0,0.0,3.0,1,2.0
2023-04-01,4,1,1,5,1.0,2.0,2.0,1,2.0
2023-04-01,5,1,1,5,0.0,5.0,3.0,1,3.0


In [31]:
XGB_result_df.to_parquet(XGB_OUTPUT_PATH)